# Model Creation

In [6]:
!pip install emoji --upgrade
!pip install pandas-profiling==2.*
!pip install plotly==4.*
#!python -m spacy download en_core_web_lg
!pip install pyldavis
!pip install gensim
!pip install chart_studio
!pip install --upgrade autopep8
!pip install regex
!pip install spacy
!pip install wordcloud
!pip install nltk

Requirement already up-to-date: emoji in c:\users\marketlytics\anaconda3\envs\finsent\lib\site-packages (0.6.0)


  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27091 sha256=070df628171444de5c8ff1c3f7f4cc81f1897023a3c237d540e50ac223fc4d53
  Stored in directory: c:\users\marketlytics\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291997 sha256=36c7e179987d9005e412f58ae7653188e5c16857e6daf2adfde357b40b33ede6
  Stored in directory: c:\users\marketlytics\appdata\local\pip\cache\wheels\cc\57\bb\6f9f52a6d8187b8f806210e3378837aa4dc6219cd64cb99846
Successfully built htmlmin imagehash
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11434 sha256=b4a45ee4c60a5c1fdeebca1cad078d28d4d7fbe9bfc99fdd1115a81d77059bea
  Stored in directory: c:\users\marketlytics\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.p

  Created wheel for autopep8: filename=autopep8-1.5.4-py2.py3-none-any.whl size=45291 sha256=2ed61b7e7f5b0d4465f5d324e2f9392a9593c3cc3fa04c95f8b033a2c2ff2f4d
  Stored in directory: c:\users\marketlytics\appdata\local\pip\cache\wheels\5f\b4\93\a30fa45720240e1106155083bac98146542dd614e1e6ce12da
Successfully built autopep8


In [26]:
import sys
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
from gensim import corpora
import pickle
import gensim
import re
from nltk.corpus import wordnet as wn
import spacy
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English
import random
from datetime import datetime
import nltk

In [39]:

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            continue
        elif token.orth_.startswith('@'):
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [token for token in tokens if token != "SCREEN_NAME"]

    return tokens



def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text


In [33]:
stopwords = set(STOPWORDS)
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
from spacy.lang.en import English
spacy.load('en')
parser = English()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marketlytics\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marketlytics\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
r_trump_tweets = requests.get('https://data-management-ac8c8.firebaseio.com/tweets/trump.json')
r_biden_tweets = requests.get('https://data-management-ac8c8.firebaseio.com/tweets/biden.json')
trump_tweets = r_trump_tweets.json()
biden_tweets = r_biden_tweets.json()
biden_df = pd.DataFrame(biden_tweets)
trump_df = pd.DataFrame(trump_tweets)

st_date_object = datetime.strptime("2020-01-01", '%Y-%m-%d')
trump_df['_date']= [datetime.strptime(bd.split(' ')[0], '%Y-%m-%d') for bd in trump_df['date']]
biden_df['_date']= [datetime.strptime(bd.split('T')[0], '%Y-%m-%d') for bd in biden_df['UTC']]

trump_df = trump_df[trump_df._date>=st_date_object]
biden_df = biden_df[biden_df._date>=st_date_object]

biden_df['month'] = pd.DatetimeIndex(biden_df['_date']).month
trump_df['month'] = pd.DatetimeIndex(trump_df['_date']).month
biden_df.columns = [x.lower() for x in biden_df.columns]
trump_df.columns = [x.lower() for x in trump_df.columns]
biden_df['hashtags']=[re.findall(r"#(\w+)",s) for s in biden_df['text']]
trump_df['hashtags']=[re.findall(r"#(\w+)",s) for s in trump_df['text']]
biden_df['users']=[re.findall(r"@(\w+)",s) for s in biden_df['text']]
trump_df['users']=[re.findall(r"@(\w+)",s) for s in trump_df['text']]

C:\Users\Marketlytics\anaconda3\envs\finsent\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [159]:
trump_df

,date,device,favorites,id,isdeleted,isretweet,retweets,text,_date,month,hashtags,users
1,2020-03-03 01:34:50,Twitter for iPhone,73748,1234653427789070300,f,f,17404,I was thrilled to be back in the Great city of...,2020-03-03,3,[KAG2020],[]
2,2020-01-17 03:22:47,Twitter for iPhone,0,1218010753434820600,f,t,7396,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17,1,[],[CBS_Herridge]
3,2020-09-12 20:10:58,Twitter for iPhone,80527,1304875170860015600,f,f,23502,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12,9,[],[]
4,2020-01-17 13:13:59,Twitter for iPhone,0,1218159531554898000,f,t,9081,RT @MZHemingway: Very friendly telling of even...,2020-01-17,1,[],[MZHemingway]
5,2020-10-23 22:04:14,Twitter for iPhone,100564,1319761576996573200,f,f,18992,THANK YOU to all of the Great American Patriot...,2020-10-23,10,[MAGA],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
54437,2020-10-23 03:42:05,Twitter for iPhone,0,1319484210101379000,f,t,9912,RT @EliseStefanik: President @realDonaldTrump ...,2020-10-23,10,[],"[EliseStefanik, realDonaldTrump]"
54438,2020-10-23 01:03:58,Twitter for iPhone,0,1319444420861829000,f,t,8249,RT @TeamTrump: LIVE: Presidential Debate #Deba...,2020-10-23,10,[Debates2020],[TeamTrump]
54439,2020-10-22 21:04:21,Twitter for iPhone,175950,1319384118849949700,f,f,36098,Just signed an order to support the workers of...,2020-10-22,10,[],[]
54440,2020-10-22 18:31:46,Twitter for iPhone,95325,1319345719829008400,f,f,19639,Suburban women want Safety &amp; Security. Joe...,2020-10-22,10,[],[]


In [42]:
import os

In [43]:
try:
    os.mkdir("models")
except:
    pass

In [151]:
def saveModel(df,user_name,i_m):


    #df = biden_df
    call_emoji_free = lambda x: give_emoji_free_text(x)

    df['emoji_free_tweets'] = df['text'].apply(call_emoji_free)

    df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)


    tokens = []

    for doc in df['url_free_tweets']:
        doc_tokens = []    
        doc_tokens = prepare_text_for_lda(doc)
        tokens.append(doc_tokens)

    # Makes tokens column
    df['tokens'] = tokens

    id2word = Dictionary(df['tokens'])

    id2word.filter_extremes(no_below=2, no_above=.99)

    corpus = [id2word.doc2bow(d) for d in df['tokens']]


    # Instantiating a Base LDA model 
    base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

    words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

    topics = [' '.join(t[0:10]) for t in words]

    # Getting the topics
    

    

    
    p=pyLDAvis.gensim.prepare(base_model, corpus, id2word)
    pyLDAvis.save_html(p, "models/"+user_name+str(i_m)+'.html')
    
    
    topic_df = p.topic_coordinates.copy()
    topic_df.sort_values(by="topic",inplace=True)
    topic_df['text']=topics
    topic_df.sort_values(by="topics",inplace=True)
    topic_df.reset_index(drop=False,inplace=True)
    topic_df['graph_topic']=topic_df['topics']
    topicDict = topic_df[['topic','graph_topic','Freq','text']].to_json(orient="records")
        
    with open("models/"+user_name+str(i_m)+'.json','w+') as f_open:
        json.dump(topicDict,f_open,indent=2)

    base_model.save("models/"+user_name+str(i_m)+'.gensim')
    
    return topics,base_model,p





# Biden Model Creation for Each Month

In [154]:
for i_month in biden_df['month'].unique():
    df = biden_df[biden_df.month==i_month]
    t,m,p = saveModel(df,'biden',i_month)

# Trump Model Creation for Each Month

In [156]:
for i_month in trump_df['month'].unique():
    df = trump_df[trump_df.month==i_month]
    t,m,p = saveModel(df,'trump',i_month)

,client,created at,favorites,hashtags,language,media type,media urls,mentions,name,retweets,...,tweet id,tweet type,urls,utc,unnamed: 16,unnamed: 17,unnamed: 18,_date,month,users
0,"<a href=""https://studio.twitter.com"" rel=""nofo...",Sat Oct 17 00:32:00 +0000 2020,5445,[],en,photo,https://pbs.twimg.com/media/EkfZWbAXIAAluo_.jpg,0,Joe Biden,1594,...,1317262047420190720,Tweet,http://iwillvote.com,2020-10-17T00:32:00.000Z,NaN,NaN,NaN,2020-10-17,10,[]
1,"<a href=""https://about.twitter.com/products/tw...",Fri Oct 16 23:54:00 +0000 2020,13160,[],en,NaN,NaN,0,Joe Biden,2846,...,1317252484071624708,Tweet,https://joebiden.com/2020/10/16/violence-again...,2020-10-16T23:54:00.000Z,NaN,NaN,NaN,2020-10-16,10,[]
2,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Fri Oct 16 23:20:00 +0000 2020,42249,[],en,NaN,NaN,0,Joe Biden,7990,...,1317243927817113600,Tweet,https://twitter.com/Yamiche/status/13172084417...,2020-10-16T23:20:00.000Z,NaN,NaN,NaN,2020-10-16,10,[]
3,"<a href=""https://periscope.tv"" rel=""nofollow"">...",Fri Oct 16 22:26:58 +0000 2020,7138,[],en,NaN,NaN,0,Joe Biden,1669,...,1317230581994811392,Tweet,https://www.pscp.tv/w/cljEMjFvUEVMQkJiT2xyS2R8...,2020-10-16T22:26:58.000Z,NaN,NaN,NaN,2020-10-16,10,[]
4,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Fri Oct 16 22:10:00 +0000 2020,11876,[],en,NaN,NaN,0,Joe Biden,3307,...,1317226311669014528,Tweet,https://twitter.com/UnivisionNews/status/13171...,2020-10-16T22:10:00.000Z,NaN,NaN,NaN,2020-10-16,10,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Thu Jan 02 02:07:41 +0000 2020,9617,[],en,NaN,NaN,0,Joe Biden,2012,...,1212556035283705858,Tweet,https://twitter.com/washingtonpost/status/1212...,2020-01-02T02:07:41.000Z,NaN,NaN,NaN,2020-01-02,1,[]
2795,"<a href=""https://studio.twitter.com"" rel=""nofo...",Thu Jan 02 01:05:00 +0000 2020,45051,[],en,video,https://video.twimg.com/amplify_video/11527637...,0,Joe Biden,10223,...,1212540258681851905,Tweet,NaN,2020-01-02T01:05:00.000Z,NaN,NaN,NaN,2020-01-02,1,[]
2796,"<a href=""https://about.twitter.com/products/tw...",Thu Jan 02 00:01:00 +0000 2020,1462,[],en,NaN,NaN,0,Joe Biden,369,...,1212524152608833536,Tweet,http://JoeBiden.com/Join,2020-01-02T00:01:00.000Z,NaN,NaN,NaN,2020-01-02,1,[]
2797,"<a href=""https://about.twitter.com/products/tw...",Wed Jan 01 18:35:00 +0000 2020,11639,[],en,NaN,NaN,0,Joe Biden,2436,...,1212442112219844609,Tweet,https://religionnews.com/2019/12/29/faith-to-r...,2020-01-01T18:35:00.000Z,NaN,NaN,NaN,2020-01-01,1,[]
